In [26]:
# 데이터 처리 
import pandas as pd 
import numpy as np
# 시각화 
import seaborn as sns 
import matplotlib as mpl
import matplotlib.pyplot as plt 
# 시각화 옵션
from IPython.display import set_matplotlib_formats
# 한글 글꼴 설정
mpl.rc('font',family ='Malgun Gothic')
# 음수값 설정 
mpl.rc('axes',unicode_minus= True )
# 그래프 선명하게 출력
set_matplotlib_formats('retina')


import pandas as pd
import numpy as np
import re

from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from sklearn.feature_extraction.text import TfidfVectorizer

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout
from tensorflow.keras import Sequential

In [27]:
from konlpy.tag import Okt
okt     = Okt()

In [28]:
train      = pd.read_csv("train_data.csv")
test       = pd.read_csv("test_data.csv")
submission = pd.read_csv("sample_submission.csv")
topic_dict = pd.read_csv("topic_dict.csv")

In [29]:
df1 = pd.read_csv('train_data.csv')
print(df1.shape)
df1.head()

(45654, 3)


,index,title,topic_idx
0,0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4
1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화,4
2,2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4
3,3,NYT 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합,4
4,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,4


In [30]:
X_train = df1['title']
Y_train = df1['topic_idx']

In [31]:
X_train[:3]

0          인천→핀란드 항공기 결항…휴가철 여행객 분통
1    실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화
2    이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것
Name: title, dtype: object

- 불용어 및 특수문자 처리

In [32]:
from konlpy.tag import Okt
import re 

In [33]:
df_stopword  = pd.read_csv('stopword.txt',header=None, names=['불용어'])
stopword_set = set(df_stopword['불용어'].values.tolist())

In [36]:
# 불용어 및 특수 기호 처리함수 구성 
def stopword_function(X): # Series 형태로 Input 
    clean_train_review = []
    okt = Okt()
    # 모든 X 값에 대해 불용어 및 특수기호 처리 
    for i in X:

        # 문자데이터에 대한 불용어 처리 
        if type(i) == str:
            # 특수 문자 처리 
            review_text = re.sub("[∼~0123456789.,()\n'→%·…①②③⑤♥곳X\n!+.,@?^↓↑㎓]" , "" ,i) 
            # Tokenizing & Stemming 
            word_text = okt.morphs(review_text,stem=True)
            # StopWord 
            word_text = [token for token in word_text if not token in stopword_set]  

            clean_train_review.append(word_text)

        # 숫자데이터가 오는 경우, 공백을 append     
        else:
            clean_train_review.append([])
            
    return clean_train_review


In [37]:
X_train_clean = stopword_function(X_train)

In [38]:
for i in X_train_clean:
    print(i)

['인천', '핀란드', '항공기', '결항', '휴가', '철', '여행객', '분통']
['실리콘밸리', '넘어서다', '구글', '조원', '들이다', '美', '전역', '거점', '화']
['이란', '외무', '긴장', '완화', '해결', '책', '은', '미국', '경제', '전쟁', '멈추다']
['NYT', '클린턴', '측근', '韓', '기업', '특수', '관계', '조명', '공과', '맞다', '물리다', '종합']
['시진핑', '트럼프', '중미', '무역', '협상', '조속', '타결', '희망']
['팔레스타인', '가자지구', '서', '세', '소년', '이스라엘군', '총격', '사망']
['인도', '만에', '파키스탄', '공습', '테러', '캠프', '폭격', '종합', '보']
['美', '대선', 'TV', '토론', '음담패설', '만회', '실패', '트럼프', '사과', '대신', '빌다', '클린턴', '공격', '하다', '역효과']
['푸틴', '한반도', '상황', '진전', '위', '한', '방안', '김정은', '위원장', '논의']
['특검', '면죄부', '받다', '트럼프', '스캔들', '보도', '언론', '맹', '공', '국민', '적']
['日', '오키나와', '서', '열리다', '강제', '징용', '노동자', '추도', '식']
['이란', '서', '최고', '지도자', '모욕', '혐의', '미국인', '징역', '선고']
['카니발', '축제', '보다', '가다', '브라질', '리우', '대형', '유람선', '행렬']
['美', '올랜도', '병원', '최악', '총기', '테러', '부상', '치료', '비', '안', '받다']
['日', '대기업', '올해', '평균', '임금', '인상']
['WMO', '엘니뇨', '여전하다', '강력', '분기', '엔', '소멸', '되다', '듯']
['이스라엘', '네타냐후', '유대교도', '병역', '문제

['날씨', '낮', '엔', '후텁', '지', '근', '장마전선', '북상', '오후', '제주', '밤', '남부', '비일']
['FC', '서울', 'K', '리그', '잔류', '성공', '부산', '연속', '승강', 'PO', '좌절']
['레알', '마드리드', '신성', '아센', '시오', '재계약']
['프로농구', 'KCC', '최', '하위', 'kt', '꺾다', '시즌', '승', '안착']
['중국', '제조', '사라지다', '中', '과학기술', '예산', '늘리다']
['아이폰', '분기', '천만천만', '대', '판매', '전망', '위', '유력']
['지브롤터', '억류', '이란', '유조선', '만에', '풀다']
['스페인', '최초', '좌파', '연정', '한', '발짝', '앞', '오늘', '표결']
['올해', '공공', '택지', '분양', '물량', '작년', '比', '감소']
['게시판', '미래부', '오늘', '안전', '한국', '훈련']
['통통', 'TV', '갤럭시', 'S', '언', '팩', '초대장', '담기다', '의미', '는']
['농구', '대표팀', '리투아니아', '클럽', '점', '차', '대승', '연승']
['러시아월드컵', '건설', '현장', '노동', '착취', '만', '연', '北', '노동자', '명', '사망']
['우', '크다', '대통령', '젤', '렌스', '키', 'EU', '대다', '제재', '계속', '유지', '돼다']
['갤럭시', '노트', '시리즈', '쏠리다', '관심']
['월드컵', '지치다', '몸', '무겁다', '마음', '떨치다', '수비', '진', '스웨덴', '방어']
['노회찬', 'BH', '가다', '흡족하다', '하다', '법원행정처', '문서', '은폐', '의혹']
['유성엽', '평화', '당', '대표', '출마', '선언', '정동영', '최경환', '파전']
['이제', '노들섬', '영화'

['한국', '당', '당', '대표', '후보', '오늘', '오후', '차', 'TV', '토론회', '격돌']
['삼성', 'SDI', '우시', '공장', '서', '태양광', '핵심', '소재', '생산', '中', '거점', '확보']
['소재', '부품', '상반기', '수출', '천', '억', '달러', '전', '比']
['월드컵', '본선', '개국', '시', '대한국', '축구', '에는', '득', '보다는', '실']
['나이', '어리다', '앵커', '만', '선호', '美', '중년', '성', '앵커', '소송']
['손시헌', '밀어내기', '볼넷', 'NC', '해커', '앞세우다', '넥센', '역전승']
['한국', '야쿠르트', '야쿠르트', '윌', '가격', '인상']
['삼성', '빅스', '비', '장터', '주', '개장', '생태계', '확장', '본격', '화']
['이동통신', '시장', '설', '대목', '실종', '번호', '이동', '평일', '수준', '못', '미치다']
['월드컵', '심판', '눈감다', '반칙', '휘슬', '더', '안타깝다', '태극', '전']
['방통위', '인터넷', '시장', '조사', '이통', '단체', '맡기다', '불공정', '우려']
['두', '손', '잡다', '정의용', '수석', '대북', '특사', '김정은', '위원장']
['한', '이란', '반세기', '⑥', '폭격', '경제', '제재', '속', '쌓다', '신뢰', '최고', '자산']
['디', '에스케이', '종', '속', '회사', '주식', '억원', '어치', '취득']
['소', '두', '증', '확산', '콜롬비아', '자메이카', '임신', '하다', '마르다']
['게시판', '캠코', '공유', '재산', '위탁', '개발', '대통령', '표창', '수상']
['美', '실리콘밸리', '서', '중국', '맥', '못', '추', '인도', '는', '승승장구

['페이스북', '청소년', '디지털', '소양', '증진', '교육', '프로그램', '공개']
['공연', '소식', '그랜드', '오페라단', '봄봄', '서울', '오페라', '앙상블', '돈', '조반', '니']
['트럼프', '시진핑', '오사카', '서', '역사', '적', '무역', '담판', '돌입', '종합', '보']
['프로축구', '부산', 'FA', '시장', '베테', '랑', '수비수', '김치우', '입']
['프로농구', '현대', '모비스', '연승', '마감', 'KCC', '완패']
['美', '인도', '카슈미르', '주민', '통제', '조속', '한', '완화', '희망']
['요시다', '만루홈런', '일본', '야구', '대표팀', '멕시코', '설욕']
['나뭇잎', '처럼', '광합성', '하다', '인공', '장치', '개발']
['난민', '송환', '타결', '모색', '중', '에도', '伊', '난민', '계속', '유입']
['최양희', '장관', 'SKT', '인수', '심사', '인위', '적', '시기', '조정', '없다']
['펠', '리페', '경복', '쌍포', '우리카드', '시즌', '승', '한국전력', '연패']
['靑', '대통령', '대면', '조사', '檢', '방침', '일정', '조만간', '입장', '정리']
['황교안', '나경원', '패스트', '트랙', '사건', '경찰', '출석', '통보', '불응', '입장']
['유엔', '개발', '계획', '내년', '대북', '사업', '중단', '계획', '없다']
['지상파', '전면', '나서다', '장외', '논객', '반갑다', '움', '우려', '교차']
['이라크', '북부', '터키', '군', '기지', '서', '유혈', '충돌', '상자', '발생']
['황', '총리', '대우조선', '롯데', 'MB', '정부', '연루', '설', '증거', '따르다', '수사']
['산학', '협력', 

['노벨', '생리', '의학상', '日', '오스미', '명예', '교수', '매우', '영광']
['콜롬비아', '대통령', '반군', '정전', '평화협정', '의회', '전달']
['KBO', '사무', '총장', 'KBOP', '분업', '야구', '산업화', '초석', '다지다']
['공사', '한창', '진행', '중인', '이집트', '신행정수도', '현장']
['일지', '성', '바이오', '직스', '분식회계', '논란']
['국정', '현안', '점검', '조정', '회의', '주재', '하다', '이낙연', '총리']
['아시안', '게임', '김학범', '호', '살림꾼', '장', '윤호', '부상', '맞다', '바꾸다']
['문', '대통령', '남북', '대화', '북핵문제', '대화', '해결', '하다', '계기', '발전', '속보']
['카카오', '자회사', '로봇', '모듈', '업체', '럭스', '보', '억원', '투자']
['이스라엘', '서', '네타냐후', '비판', '대규모', '시위', '기소', '면제', '안', '돼다']
['北', '매체', '홍수', '피해', '무산', '광산', '조업', '재개', '보도']
['러시아', '美', '중거리핵전력조약', '이행', '중단', '결정', '강력', '비난']
['두산', 'LG', '전', '연', '승두산', '은', '우승', '하다', '발', '더', 'LG', '늘다']
['방통위', '추천', 'KBS', '이사', '부적', '격자', '포함']
['임종룡', '공매도', '공시', '제도', '개선', '하다']
['KT', 'Y', '수능', '데이터', '무제한', '페스티벌']
['이란', '핵', '합의', '이행', '축소', '단계', 'D', '유럽', '약속', '제대로', '지키다']
['남태평양', '바누아투', '규모', '지진', '발생']
['따뜻하다', '입다', '출근', '하다', '전북', '개', 

...

['증권', '파생상품', '시장', '폐장', '식']
['태양광', '전', '기', '생산', '하다', '빛', '내다', '발광', '전지', '개발', '성공']
['갤럭시', 'S', '카메라', '성능', '해외', '평가', '기관', '서', '역대', '최고점']
['朴', '대통령', '北', '발사', '만에', 'NSC', '靑', '액션', '플랜', '대응', '종합']
['코스피', '선', '근접']
['신림동', '강간미수', 'SNS', '상속', '남성', '체포', '주거침입', '혐의']
['신간', '삶다', '계속', '되다']
['영상', '류현진', '베테', '랑', '포수', '마틴', '출격', '워싱턴', '은']
['춘천시', '지역', '대학', '손잡다', '상생', '협력', '협의', '회', '추진']
['유가', '꺾다', '수입', '물가', '지수', '개', '만에', '최고']
['그래픽', '정부', '개헌안', '준비', '일정']
['SK', '텔레콤', '문자', '인증', '없이', '본인', '확인', 'T', '인증', '출시']
['중국인', '부정', '적', '태도', '뉴질랜드', '관광', '악', '영향', '경고']
['북', '수소탄', '실험', '국방부', '증폭', '핵분열', '추정']
['공개', 'D', '개다', '노트', '년전', '불명예', '씻다', '전세계', '주목']
['이란', '핵', '합의', '의무', '일부', '유예', '개시', '우라늄', '중수', '한도', '넘기다']
['행정부', '멈추다', '국회', '는', '차기', '권력', '셈법', '만', '위기', '리더십', '실종']
['제주', '서', '다문화', '가정', '직장인', '성공', '신화', '시트콤', '방송', '종합']
['국고', '채', '금리', '일제', '히', '하락', '년물', '연']
['가다', '여름', '아쉽다', '

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



['피아니스트', '조지', '리', '워즈워스', '詩', '음악', '더', '깊다', '이해', '가능']
['공공기관', '직무', '급제', '도입', '노력', '사회', '적', '합의', '임금', '개편', '신호탄', '종합']
['순천향대', '코로나', '특별', '장학금', '지급', '실제', '납입', '수업료']
['코로나', '확산', '와중', '편입', '시험', '줄줄이', '대학가', '방역', '긴장']
['대전', '교육청', '대한민국', 'SNS', '대상', '교육', '기관', '부문', '대상']
['등교', '하다', '학생', '반기다', '교직원']
['쌍용차', '중남미', '시장', '공략', '강화', '코란도', '신차', '출시', '마케팅', '행사']
['러', '해킹', '그룹', '테베스', '축구선수', '치료', '약물', '사용', '기록', '폭로']
['전북대', '한', '옥', '대', '목수', '양성', '과정', '교육', '생', '모집']
['방송', '문화', '진흥', '회', '이완', '기', '이사장', '이사장', '직', '사임']
['국제', '금값', '하락', 'KR', '금값', '도', '급락', '종합']
['메시', '멀티골', '바르사', '레가네스', '꺾다', '위', '탈환']
['伊', '밀라노', '호스텔', '외국인', '만', '할인', '反', '난민', '법', '반발']
['스티븐', '로치', '밉다', '더블딥', '확률', '넘다', '약', '달러', '경고']
['교촌치킨', '교촌', '에프', '앤비', '코스피', '신규', '상장']
['코스피', '밉다', '연', '준', '부양책', '반등', '출발', '대', '회복', '종합']
['한상혁', '방통위', '원장', '소모', '적', 'G', '마케팅', '과열', '경쟁', '지', '양해']
['호캉스', '도', '아빠', '챙기다', '주'

['잠적', '하다', '伊', '테러리스트', '볼리비아', '서', '체포', '주', '송환', '되다', '듯']
['기', '정통부', '사물인터넷', '보안', '인증', '제도', '개선', '연구', '반', '가동']
['원', '비', '빼', '돌리다', '명품', '백', '살다', '유치원', '장', '처벌', '가능', '유치원', '법', '통과']
['월드컵', '목발', '치우다', '기성용', '멍', '든', '루디', '훈련', '동행', '벼']
['이다', '우리은행', '도', '라임', '펀드', '원금', '전액', '반환', '수용', '미루다']
['수업', '일수', '감축', '피해', '대책', '마련', '라']
['IBS', '프랑스', '연구', '팀', '유리', '입자', '끼리', '임계점', '뭉치다', '케이', '지', '형성', '과정', '첫', '규명']
['네팔', '안나푸르나', '눈사태', '충남', '봉사', '단', '교사', '실종', '발견']
['문', '대통령', '개회', '식', '리셉션', '서', '참가', '국', '정상', '평창', '다자', '외교']
['조성진', '김선욱', '길트', '부르다', '크다', '라흐마니노프', '는', '새해']
['놀런', '에러', '나다', '류현진', '천적', '나다', '이유', '모르다']
['중간', '신영석', '주장', '위아래', '똘똘', '뭉치다', '남자배구', '대표팀']
['병상', '부족', '상주', '이송', '되다', '대구', '확', '진자']
['배달', '종사', '산재', '보험', '적용', '확대', '노', '사정', '합의', '마련']
['유가', '증권', '시장', '공시', '우수', '법인', 'LG', '유플러스', '개사', '선정']
['과학기술', '자상', '변재', '형', 'KAIST', '교수']
['제주', '서귀포', '서', '버스', '끼

['연례', '행사', '서', '월례', '행사', '되다', '美', '함정', '대만해협', '통과', '中', '반발', '종합', '보']
['부산', '청년', '취업', '지원', '관', '시행', '맞춤', '형', '구직', '상담']
['진상', '조', '사위', '합의', '이행', '라', '이재학', 'PD', '대책위', '박일', '집회']
['에든버러', '축제', '참가', '작', '단돈', '천원', '즐기다', '보다']
['아자', '디', '광장', '열리다', '이란', '이슬람', '혁명', '주년', '대규모', '집회']
['특징', '주', '크다', '바하이', '텍', '대규모', '증자', '소식', '급락']
['파', '다르다', 'vs', '정지', '석', '배구', '올스타전', '서', '대포', '알', '서브', '왕', '도전']
['朴', '대통령', '만에', '예비군', '격려', '카드', '총력', '안보', '구심', '점']
['회', '해킹', '방어', '대회', '대상', '올가미', '팀']
['신간', '밉다', '욕', '보이', '다', '파사드', '서울']
['광고', '비', '음식', '인터넷', '언론사', '횡포', '공무원', '분노']
['고교', '학력평가', '주관', '교육청', '서', '광주', '전남', '빠지다', '이유', '는']
['정밀', '진단', '플랫폼', '기업', '엔', '젠', '바이오', '코스닥', '입성']
['프로농구', '강', '윤호영', '양', '희종', '김선형', '전태풍', '만은', '이기다']
['영흥화력발전소', '서', '숨지다', '운전기사', '결식']
['엘에스', '렉', '트릭', '분기', '영업', '이익', '억원', '작년', '동기', '대비']
['보', '이주열', '금융시장', '악화', '시', '비', '은행', '금융기관', '대출', '검토']
['가짜', '뉴

In [39]:
X_train_clean[:3]

[['인천', '핀란드', '항공기', '결항', '휴가', '철', '여행객', '분통'],
 ['실리콘밸리', '넘어서다', '구글', '조원', '들이다', '美', '전역', '거점', '화'],
 ['이란', '외무', '긴장', '완화', '해결', '책', '은', '미국', '경제', '전쟁', '멈추다']]

In [40]:
X_length = X_train.apply(len)
X_length.describe()

# 대략 20 으로 잡는걸로 결정

count    45654.000000
mean        27.330552
std          4.968172
min          4.000000
25%         25.000000
50%         28.000000
75%         31.000000
max         44.000000
Name: title, dtype: float64

In [41]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [42]:
model_token = Tokenizer()
model_token.fit_on_texts(X_train_clean)

In [43]:
model_token

In [44]:
# Embedding Layer 구성 작업, Input Unit  
len(model_token.word_index)

28001

In [45]:
X_train_clean

[['인천', '핀란드', '항공기', '결항', '휴가', '철', '여행객', '분통'],
 ['실리콘밸리', '넘어서다', '구글', '조원', '들이다', '美', '전역', '거점', '화'],
 ['이란', '외무', '긴장', '완화', '해결', '책', '은', '미국', '경제', '전쟁', '멈추다'],
 ['NYT', '클린턴', '측근', '韓', '기업', '특수', '관계', '조명', '공과', '맞다', '물리다', '종합'],
 ['시진핑', '트럼프', '중미', '무역', '협상', '조속', '타결', '희망'],
 ['팔레스타인', '가자지구', '서', '세', '소년', '이스라엘군', '총격', '사망'],
 ['인도', '만에', '파키스탄', '공습', '테러', '캠프', '폭격', '종합', '보'],
 ['美',
  '대선',
  'TV',
  '토론',
  '음담패설',
  '만회',
  '실패',
  '트럼프',
  '사과',
  '대신',
  '빌다',
  '클린턴',
  '공격',
  '하다',
  '역효과'],
 ['푸틴', '한반도', '상황', '진전', '위', '한', '방안', '김정은', '위원장', '논의'],
 ['특검', '면죄부', '받다', '트럼프', '스캔들', '보도', '언론', '맹', '공', '국민', '적'],
 ['日', '오키나와', '서', '열리다', '강제', '징용', '노동자', '추도', '식'],
 ['이란', '서', '최고', '지도자', '모욕', '혐의', '미국인', '징역', '선고'],
 ['카니발', '축제', '보다', '가다', '브라질', '리우', '대형', '유람선', '행렬'],
 ['美', '올랜도', '병원', '최악', '총기', '테러', '부상', '치료', '비', '안', '받다'],
 ['日', '대기업', '올해', '평균', '임금', '인상'],
 ['WMO', '엘니뇨', '여전하다', '강력', '분기

In [46]:
X_sequence = model_token.texts_to_sequences(X_train_clean)

max_length = 45
train_x    = pad_sequences(X_sequence, maxlen=max_length, padding='post')

In [47]:
train_x

array([[ 373, 5054, 2579, ...,    0,    0,    0],
       [5719, 4500,  349, ...,    0,    0,    0],
       [  21,  699,  949, ...,    0,    0,    0],
       ...,
       [  33,  927,   81, ...,    0,    0,    0],
       [3126,    1,  255, ...,    0,    0,    0],
       [  10,  338,  257, ...,    0,    0,    0]])

In [48]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras        import layers
from tensorflow.keras.optimizers import RMSprop, SGD
from tensorflow.keras.utils  import to_categorical

# Random Seed 

tensorflow.random.set_seed(7)

In [49]:
# Y 값을 Matrix 형태로 변환
train_y = to_categorical(Y_train.astype(int))

- Test Set 데이터 분류 실시 

In [50]:
# 데이터 불러오기 
df_test = pd.read_csv("test_data.csv")
print(df_test.shape)
df_test.head(2)

(9131, 2)


,index,title
0,45654,유튜브 내달 2일까지 크리에이터 지원 공간 운영
1,45655,어버이날 맑다가 흐려져…남부지방 옅은 황사


In [51]:
# 특수문자처리 / 불용어 처리 
X_test_clean = stopword_function(df_test['title'])

In [52]:
X_test_clean

[['유튜브', '내달', '크리에이터', '지원', '공간', '운영'],
 ['어버이날', '맑다', '흐려지다', '남부', '지방', '옅다', '황사'],
 ['내년', '국가', 'RD', '평가', '논문', '건수', '는', '반영', '않다'],
 ['김명자', '신임', '과총', '회장', '원로', '젊다', '과학자', '지혜', '모으다'],
 ['회색', '인간', '작가', '김', '동식', '양심', '고백', '새', '소설', '집', '권', '추다', '간'],
 ['야외', '서', '생방송', '하다', '액션', '캠', '전용', '요', '금제', '잇따르다'],
 ['월드컵', '태극전사', '강', '전', '초기', '지', '레오', '강', '입성', '종합'],
 ['미세먼지', '속', '출근길'],
 ['왓츠앱', '稅', '원', '성나다', '레바논', '민심', '총리', '사퇴', '이어지다', '종합', '보'],
 ['베트남', '경제', '고', '성장', '지속', '분기', 'GDP', '성장'],
 ['그리스', '서', '한', '국전', '참전', '기념', '식', '참전용사', '한반도', '평화', '기원'],
 ['정진석', '이정현', '당', '현실', '냉정하다', '보다', '물러나다', '좋다', '종합'],
 ['美', '베네수엘라', '구호', '품', '반입', '촉구', '안보리', '결의', '추진'],
 ['황재균', '쐐기', '타', 'kt', '갈다', '길', '바쁘다', '삼성', '고춧가루'],
 ['정상회담', 'D', '文', '대통령', '취임', '후', '남북관계', '주요', '일지'],
 ['LGU', '＋', '분기', '영업', '익', '천억원', '마케팅', '비용', '감소', '종합'],
 ['박원순', '시장', '아부다비', '루브르', '박물관', '방문'],
 ['방심위', '강릉', '서', '고성', '산

In [53]:
#X_test_clean

In [54]:
# Text to Sequence / Padding 
X_test_sequence = model_token.texts_to_sequences(X_test_clean)
test_x    = pad_sequences(X_test_sequence,maxlen=max_length, padding='post')

In [55]:
# 전처리 완료

- Model

In [56]:
es = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3,
                   verbose=1, mode='min', baseline=None, restore_best_weights=True)

In [63]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Conv1D, MaxPooling1D
from tensorflow.keras.datasets import imdb
 

In [72]:
# seed=7
# model = Sequential()
# model.add(Embedding(5000, 100))
# model.add(Dropout(0.5))
# model.add(Conv1D(64, 5, padding='valid', activation='relu',strides=1))
# model.add(MaxPooling1D(pool_size=4))
# model.add(LSTM(55))
# model.add(Dense(1))
# model.add(layers.Dense(7, activation='softmax'))
  
# # 모델의 컴파일


# model.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['acc'])



seed=7

model_lstm = Sequential()

model_lstm.add(layers.Embedding(31000,128, input_length=max_length))

model_lstm.add(Dropout(0.03)) # ADDED

model_lstm.add(Conv1D(64, 5))

model_lstm.add(layers.Bidirectional(layers.LSTM(32)))


model_lstm.add(Dropout(0.2)) # ADDED




model_lstm.add(layers.Dense(7, activation='softmax'))


model_lstm.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['acc'])

model_lstm.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 45, 128)           3968000   
_________________________________________________________________
dropout_14 (Dropout)         (None, 45, 128)           0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 41, 64)            41024     
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 64)                24832     
_________________________________________________________________
dropout_15 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 7)                 455       
Total params: 4,034,311
Trainable params: 4,034,311
Non-trainable params: 0
___________________________________________

In [73]:
history = model_lstm.fit(train_x, train_y,
                        epochs=8, batch_size=180, validation_split=0.2, callbacks=es)

Epoch 1/8
203/203 [==============================] - 18s 36ms/step - loss: 0.9794 - acc: 0.6543 - val_loss: 0.8111 - val_acc: 0.7237
Epoch 2/8
203/203 [==============================] - 5s 23ms/step - loss: 0.4334 - acc: 0.8651 - val_loss: 0.6454 - val_acc: 0.7841
Epoch 3/8
203/203 [==============================] - 5s 23ms/step - loss: 0.3274 - acc: 0.8979 - val_loss: 0.6930 - val_acc: 0.7602
Epoch 4/8
203/203 [==============================] - 5s 23ms/step - loss: 0.2669 - acc: 0.9167 - val_loss: 0.6497 - val_acc: 0.7768
Epoch 5/8
203/203 [==============================] - 5s 22ms/step - loss: 0.2216 - acc: 0.9322 - val_loss: 0.6749 - val_acc: 0.7843
Restoring model weights from the end of the best epoch.
Epoch 00005: early stopping


In [74]:
# 계층 교차 검증
n_fold = 5
seed = 21

cv = StratifiedKFold(n_splits = n_fold, shuffle=True, random_state=seed)

# 테스트데이터의 예측값 담을 곳 생성
test_y = np.zeros((test_x.shape[0], 7))

# 조기 종료 옵션 추가

es = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3,
                   verbose=1, mode='min', baseline=None, restore_best_weights=True)

for i, (i_trn, i_val) in enumerate(cv.split(train_x, Y_train), 1):
    print(f'training model for CV #{i}')

    model_lstm.fit(train_x[i_trn], 
            to_categorical(Y_train[i_trn]),
            validation_data=(train_x[i_val], to_categorical(Y_train[i_val])),
            epochs=20,
            batch_size=180,
            callbacks=[es])     # 조기 종료 옵션
                      
    test_y += model_lstm.predict(test_x) / n_fold    # 나온 예측값들을 교차 검증 횟수로 나눈다

training model for CV #1
Epoch 1/20
203/203 [==============================] - 5s 25ms/step - loss: 0.3809 - acc: 0.8794 - val_loss: 0.3644 - val_acc: 0.8803
Epoch 2/20
203/203 [==============================] - 5s 23ms/step - loss: 0.2916 - acc: 0.9080 - val_loss: 0.3858 - val_acc: 0.8761
Epoch 3/20
203/203 [==============================] - 5s 23ms/step - loss: 0.2387 - acc: 0.9264 - val_loss: 0.4108 - val_acc: 0.8703
Epoch 4/20
203/203 [==============================] - 5s 23ms/step - loss: 0.1971 - acc: 0.9400 - val_loss: 0.4385 - val_acc: 0.8678
Restoring model weights from the end of the best epoch.
Epoch 00004: early stopping
training model for CV #2
Epoch 1/20
203/203 [==============================] - 5s 23ms/step - loss: 0.3193 - acc: 0.8992 - val_loss: 0.2771 - val_acc: 0.9083
Epoch 2/20
203/203 [==============================] - 5s 23ms/step - loss: 0.2596 - acc: 0.9191 - val_loss: 0.3027 - val_acc: 0.9022
Epoch 3/20
203/203 [==============================] - 5s 23ms/step -

In [75]:
# 테스트 데이터의 예측값 확인
test_y

array([[4.52554338e-02, 4.78343321e-02, 2.62572093e-01, ...,
        2.58825548e-03, 3.06501324e-03, 2.66316006e-03],
       [1.76288226e-04, 8.21163048e-04, 2.79914431e-03, ...,
        2.61854884e-04, 2.52520790e-04, 1.90691206e-04],
       [4.74067819e-02, 9.55299130e-02, 7.36302108e-01, ...,
        4.70234186e-03, 1.58280933e-03, 1.08721060e-01],
       ...,
       [2.99701747e-03, 7.19514518e-03, 8.21362366e-02, ...,
        1.45038354e-02, 5.75608959e-03, 9.75765543e-04],
       [5.86905375e-01, 1.07751397e-01, 2.68449150e-01, ...,
        2.63281632e-02, 2.95924765e-03, 4.73737979e-03],
       [4.00736404e-03, 1.47830229e-02, 3.87360476e-01, ...,
        7.79231585e-03, 3.74187651e-03, 5.67154810e-01]])

In [76]:
topic = []
for i in range(len(test_y)):
    topic.append(np.argmax(test_y[i]))

In [77]:
# sample_submission 에 대입
submission['topic_idx'] = topic
submission['topic_idx'].value_counts()

4    1516
6    1444
5    1428
2    1221
3    1220
1    1204
0    1098
Name: topic_idx, dtype: int64

In [78]:
submission.to_csv('lstm_rnn.csv',index = False)